In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import time
import re

In [ ]:
links = pd.read_csv('car_links', skiprows=1) #Indlæser CSV fil der indeholder links
features_to_check = [
    'GPS navigation', 'Aircon/Klima', 'Anhængertræk', '4-hjulstræk', 'Parkeringssensor',
    'Fartpilot', 'ABS', 'Alarm', 'Alufælge', 'Antispin', 'Bluetooth', 'Centrallås',
    'ESP', 'El-ruder', 'El-spejle', 'El-sæder', 'Fuldlæder', 'Glastag', 'Headup display',
    'Isofix', 'Klimaanlæg', 'Kurvelys', 'Kørecomputer', 'Multifunktionsrat', 'Regnsensor',
    'Servo', 'Soltag', 'Sportspakke', 'Sportsæder', 'Stop&Go', 'Sædevarme', 'Vejbane advarselssystem',
    'Xenon lygter', 'Ikke-ryger', 'Én-ejers', 'Service OK', 'Nysynet', 'Demo Bil', 'Sportssæder',
    'Dellæder', 'CVR/Engros', 'Fuldlæder', 'Adaptiv fartpilot', 'Automatgear', 'Bakkamera', 'Blindvinkelassistent',
    'Digital instrumentering', 'Elektrisk bagklap'] # De features vi vil tjekke om bilerne har

car_data = [] # Laver en tom liste til at gemme data i

feature_presence = {feature: 'No' for feature in features_to_check} #Vi laver en dictionary med features og sætter dem til 'No' som default

for url in links[links.columns[0]].values: #Vi looper igennem alle links i vores CSV fil
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    car_headers = soup.find('div', class_='car-specs__pictures-and-car-data-container') # Søger efter den frame hvor den "primære" data er
    car_row = [] # Resetter vores array
    model = soup.find('h1', class_='car-headline').get_text().strip() #Finder der hvor modelnavnet står
    car_row.append(f"{model}") # Gemmer modelnavnet
    if car_headers:
        datas = car_headers.findChildren('div', class_='car-data__group-info')
        for data in datas: # Looper igennem den "primære" data og appender det vi finder til vores array
            span_text = data.find('span')
            if span_text:
                model_text = span_text.get_text().strip() 
                car_row.append(model_text)
    sales_price = soup.find('span', class_='price right').get_text().strip() # Finder salgsprisen
    numbers = re.findall(r'\d+', sales_price) # Splitter teksten op så der ikke står kr. foran
    cleaned_price = '.'.join(numbers) # Joiner tallene
    car_row.append(f"{cleaned_price}") # Gemmer tallene
    car_equipment_container = soup.find('div', class_='car-specs__equipment-container')
    if car_equipment_container:
        equipment_lists = car_equipment_container.find_all('ul', class_='equipment-list') # Finder der hvor alle features står
        for equipment_list in equipment_lists:
            list_items = equipment_list.find_all('li')
            for item in list_items:
                span = item.find('span')
                if span and 'Farve:' in span.get_text(): #Tjekker og gemmer farven
                    color = item.get_text().split(':')[1].strip()
                    car_row.append(color) 
                for feature in features_to_check: 
                    if feature in item.get_text():
                        feature_presence[feature] = 'Yes' # Hvis featuren er i listen, så sætter vi den til Yes frem for No
        for feature, presence in feature_presence.items():
            car_row.append(f"{presence}") # Gemmer om bilen har featuren eller ej
    car_row.append(url) # Gemmer URL'en så vi har noget at linke tilbage til hvis der skulle være noget galt med dataen
    np.transpose(car_row) # Flipper dataen så det står vandret frem for lodret
    car_data.append(car_row) # Gemmer til vores tomme liste vi lavede i starten
    time.sleep(4) # Sleep timer så vi ikke får too many requests/timeout
df = pd.DataFrame(car_data) #Vi gemmer vores liste i en dataframe
df.to_csv('car_data', index=False)  # Gemmer vores csv fil når loopet er done, tog ca. 24 timer at køre igennem 20.000 links